In [1]:
import pandas as pd
import glob
import os

## Import and consolidate csv for SPY

In [87]:
import tqdm

path = R"X:\Projects\SPY Data\Data\Consol\cmd_csv"
all_files = glob.iglob(os.path.join(path, "*.csv"))
li = []
col_names = ["ID", "Ticker","Expiration","Ask","Asize","Bid","Bsize","Last","Type","Strike","Vol","OpenInterest","Underlying","DataDate"]

for f in tqdm.notebook.tqdm(all_files):
    try:
        df = pd.read_csv(f, index_col=None, skiprows=1, names=col_names, parse_dates=["Expiration", "DataDate"])
        df = df[df["Ticker"]=="SPY"]
        li.append(df)
        print("Finished " + str(f))
    except:
        print("NO SPY NO SPY" + str(f))

cts\SPY Data\Data\Consol\cmd_csv\D_20201104_OData1.csv
Finished X:\Projects\SPY Data\Data\Consol\cmd_csv\D_20201104_OData2.csv
Finished X:\Projects\SPY Data\Data\Consol\cmd_csv\D_20201105_OData1.csv
Finished X:\Projects\SPY Data\Data\Consol\cmd_csv\D_20201105_OData2.csv
Finished X:\Projects\SPY Data\Data\Consol\cmd_csv\D_20201106_OData1.csv
Finished X:\Projects\SPY Data\Data\Consol\cmd_csv\D_20201106_OData2.csv
Finished X:\Projects\SPY Data\Data\Consol\cmd_csv\D_20201109_OData1.csv
Finished X:\Projects\SPY Data\Data\Consol\cmd_csv\D_20201109_OData2.csv
Finished X:\Projects\SPY Data\Data\Consol\cmd_csv\D_20201110_OData1.csv
Finished X:\Projects\SPY Data\Data\Consol\cmd_csv\D_20201110_OData2.csv
Finished X:\Projects\SPY Data\Data\Consol\cmd_csv\D_20201111_OData1.csv
Finished X:\Projects\SPY Data\Data\Consol\cmd_csv\D_20201111_OData2.csv
Finished X:\Projects\SPY Data\Data\Consol\cmd_csv\D_20201112_OData1.csv
Finished X:\Projects\SPY Data\Data\Consol\cmd_csv\D_20201112_OData2.csv
Finished 

In [88]:
# join all the csvs together once consolidated

frame = pd.concat(li, axis=0, ignore_index = True)

## Reading the pickle file

In [2]:
frame = pd.read_pickle("SPY_options.pickle")

In [3]:
from datetime import timedelta
from datetime import date

# create a truncated test kit

truncated = frame.head(n = 100000)
truncated["DataDate"] = pd.to_datetime(truncated["DataDate"])
truncated = truncated.sort_values(by="DataDate").reset_index()
truncated["DTE"] = truncated["Expiration"]- truncated["DataDate"]
truncated[["oid", "iddate"]] = truncated["ID"].str.split(pat=".", expand=True)

<ipython-input-3-d019f961a97e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  truncated["DataDate"] = pd.to_datetime(truncated["DataDate"])


In [10]:
truncated[(truncated["DTE"] == timedelta(currDTE)) & (truncated["Strike"] == 288) & (truncated["Type"] == "put")]

,index,ID,Ticker,Expiration,Ask,Asize,Bid,Bsize,Last,Type,Strike,Vol,OpenInterest,Underlying,DataDate,DTE
11501,21990,SPY2019-06-14P288.002019-06-07,SPY,2019-06-14,2.35,1,2.34,269,2.35,put,288.0,11647,3584,287.65,2019-06-07,7 days
17572,29915,SPY2019-06-17P288.002019-06-10,SPY,2019-06-17,1.62,500,1.59,303,1.60,put,288.0,3368,586,288.97,2019-06-10,7 days
34258,45160,SPY2019-06-19P288.002019-06-12,SPY,2019-06-19,1.91,237,1.88,180,2.00,put,288.0,7101,5233,288.39,2019-06-12,7 days
45770,61338,SPY2019-06-21P288.002019-06-14,SPY,2019-06-21,1.90,122,1.88,275,1.89,put,288.0,11547,45986,289.26,2019-06-14,7 days
52945,68948,SPY2019-06-24P288.002019-06-17,SPY,2019-06-24,1.89,309,1.86,100,1.99,put,288.0,393,661,289.37,2019-06-17,7 days
72326,83815,SPY2019-06-26P288.002019-06-19,SPY,2019-06-26,0.70,140,0.69,889,0.65,put,288.0,1361,2510,293.06,2019-06-19,7 days
85144,94498,SPY2019-07-01P288.002019-06-24,SPY,2019-07-01,0.85,394,0.83,951,0.89,put,288.0,941,4437,293.64,2019-06-24,7 days
91690,543,SPY2019-08-07p288.002019-07-31,SPY,2019-08-07,0.66,417,0.64,888,0.60,put,288.0,981,492,297.43,2019-07-31,7 days
98169,7009,SPY2019-10-07p288.002019-09-30,SPY,2019-10-07,0.29,0,1870.00,339,0.34,put,288.0,662,831,296.77,2019-09-30,7 days


In [121]:
curr_trade = truncated[(truncated["DataDate"] == truncated["DataDate"].min() + timedelta(1)) & (truncated["Strike"] == 288) & (truncated["Type"] == "put") & (truncated["DTE"] < timedelta(5)) & (truncated["DTE"] > timedelta(1))]["oid"].values[0]

In [122]:
truncated[truncated["oid"] == curr_trade]

,index,ID,Ticker,Expiration,Ask,Asize,Bid,Bsize,Last,Type,Strike,Vol,OpenInterest,Underlying,DataDate,DTE,oid,iddate
4712,14196,SPY2019-06-10P288.002019-06-06,SPY,2019-06-10,3.60,340,3.40,380,3.11,put,288.0,37,23,284.80,2019-06-06,4 days,SPY2019-06-10P288,002019-06-06
11353,21506,SPY2019-06-10P288.002019-06-07,SPY,2019-06-10,1.31,349,1.28,55,1.30,put,288.0,36598,45,287.65,2019-06-07,3 days,SPY2019-06-10P288,002019-06-07
14648,28924,SPY2019-06-10P288.002019-06-10,SPY,2019-06-10,0.01,4475,0.00,0,0.01,put,288.0,50018,8721,288.97,2019-06-10,0 days,SPY2019-06-10P288,002019-06-10
27328,36130,SPY2019-06-10P288.002019-06-11,SPY,2019-06-10,0.01,3403,0.00,0,0.01,put,288.0,0,14238,288.90,2019-06-11,-1 days,SPY2019-06-10P288,002019-06-11
29824,43552,SPY2019-06-10P288.002019-06-12,SPY,2019-06-10,0.01,3403,0.00,0,0.01,put,288.0,0,14238,288.39,2019-06-12,-2 days,SPY2019-06-10P288,002019-06-12
40804,51198,SPY2019-06-10P288.002019-06-13,SPY,2019-06-10,0.01,3403,0.00,0,0.01,put,288.0,0,14238,289.58,2019-06-13,-3 days,SPY2019-06-10P288,002019-06-13
49947,58688,SPY2019-06-10P288.002019-06-14,SPY,2019-06-10,0.01,3403,0.00,0,0.01,put,288.0,0,14238,289.26,2019-06-14,-4 days,SPY2019-06-10P288,002019-06-14
55897,66050,SPY2019-06-10P288.002019-06-17,SPY,2019-06-10,0.01,3403,0.00,0,0.01,put,288.0,0,14238,289.37,2019-06-17,-7 days,SPY2019-06-10P288,002019-06-17
61244,73220,SPY2019-06-10P288.002019-06-18,SPY,2019-06-10,0.01,3403,0.00,0,0.01,put,288.0,0,14238,292.40,2019-06-18,-8 days,SPY2019-06-10P288,002019-06-18
71248,80507,SPY2019-06-10P288.002019-06-19,SPY,2019-06-10,0.01,3403,0.00,0,0.01,put,288.0,0,14238,293.06,2019-06-19,-9 days,SPY2019-06-10P288,002019-06-19


## Testing backtesting functions

In [2]:
from fuctions import Trade, AccountBal, TradeOperations
from datetime import timedelta
from datetime import date

truncated = pd.read_pickle("truncated.pickle")

In [7]:
curr_trade_open = Trade("SPY2019-06-10P288", 3.40, date(2019,6,10))
curr_trade_close = Trade("SPY2019-06-10P288", 0.40, date(2019,6,10))
next_trade = Trade("SPY2019-06-17P290", 2.40, date(2019,6,17))
my_account = AccountBal(100)

In [3]:
start_date = truncated["DataDate"].min()

In [31]:
truncated[truncated["DataDate"] == start_date]["Underlying"][0]

284.8

In [8]:
start_date = truncated["DataDate"].min()

def run_backtest(curr_date, dataset):

    curr_account = AccountBal(100)
    tradeOn = False

    while curr_date < dataset["DataDate"].max(): # loop through the days
        curr_date += timedelta(1)
        daily_set = dataset[dataset["DataDate"] == curr_date]
        if tradeOn == False: # if there is no trade on, we want to search for a trade
            new_trade = Trade(TradeOperations.find_trade(dataset)) # create new trade class
            tradeOn = True
            curr_account.add_trade(new_trade.get_value())
            print(curr_account.trade_val)
            
        elif tradeOn == Trade:
            pass
                    

In [9]:
run_backtest(start_date, truncated)

[2.34]


In [6]:
TradeOperations.find_trade(sample_day)

'SPY2019-06-14P285'

In [3]:
sample_day = truncated[truncated["DataDate"] == truncated["DataDate"].min()]

In [5]:
TradeOperations.find_trade(sample_day)[0]

'SPY2019-06-14P285'

In [22]:
sample_day

,index,ID,Ticker,Expiration,Ask,Asize,Bid,Bsize,Last,Type,Strike,Vol,OpenInterest,Underlying,DataDate,DTE,oid,iddate
0,15671,SPY2019-12-20C130.002019-06-06,SPY,2019-12-20,155.03,37,154.72,37,118.66,call,130.0,0,11,284.8,2019-06-06,197 days,SPY2019-12-20C130,002019-06-06
1,17942,SPY2019-12-31P309.002019-06-06,SPY,2019-12-31,27.14,37,26.87,152,0.00,put,309.0,0,0,284.8,2019-06-06,208 days,SPY2019-12-31P309,002019-06-06
2,17941,SPY2019-12-31P308.002019-06-06,SPY,2019-12-31,26.35,113,26.08,152,21.35,put,308.0,0,25,284.8,2019-06-06,208 days,SPY2019-12-31P308,002019-06-06
3,17940,SPY2019-12-31P307.002019-06-06,SPY,2019-12-31,25.58,113,25.32,314,33.00,put,307.0,0,0,284.8,2019-06-06,208 days,SPY2019-12-31P307,002019-06-06
4,17939,SPY2019-12-31P306.002019-06-06,SPY,2019-12-31,24.83,169,24.55,153,0.00,put,306.0,0,0,284.8,2019-06-06,208 days,SPY2019-12-31P306,002019-06-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7295,15495,SPY2019-07-19P310.002019-06-06,SPY,2019-07-19,26.50,251,26.30,338,27.79,put,310.0,1001,120,284.8,2019-06-06,43 days,SPY2019-07-19P310,002019-06-06
7296,15494,SPY2019-07-19P309.002019-06-06,SPY,2019-07-19,25.53,251,25.25,251,26.24,put,309.0,0,15,284.8,2019-06-06,43 days,SPY2019-07-19P309,002019-06-06
7297,15492,SPY2019-07-19P307.002019-06-06,SPY,2019-07-19,23.54,251,23.27,251,19.10,put,307.0,0,11,284.8,2019-06-06,43 days,SPY2019-07-19P307,002019-06-06
7298,14641,SPY2019-06-14P264.002019-06-06,SPY,2019-06-14,0.04,2010,0.03,4520,0.04,put,264.0,303,4215,284.8,2019-06-06,8 days,SPY2019-06-14P264,002019-06-06
